In [1]:
import numpy as np

In [113]:
def load_data(input_id=''): 
    X = np.loadtxt("input"+input_id+".txt", dtype='str')
    X = np.array([list(x) for x in X])
    return X


def search_start(X):
    r, c = np.where(X=='S')
    return [r[0], c[0]]


def connected_tiles(X, t):
    r, c = t
    symbol = X[r][c]
    north = [r-1, c]
    east  = [r, c+1]
    south = [r+1, c]
    west  = [r, c-1]
    if symbol == 'S':
        tiles = [north, east, south, west]
    elif symbol == '|':
        tiles = [north, south]
    elif symbol == '-':
        tiles = [west, east]
    elif symbol == 'L':
        tiles = [north, east]
    elif symbol == 'J':
        tiles = [north, west]
    elif symbol == '7':
        tiles = [south, west]
    elif symbol == 'F':
        tiles = [south, east]
    else:
        tiles = []
    return tiles


def is_connected(tile1, tile2):
    ''' if two tiles are connected by pipe '''
    result = False 
    if (tile1 in connected_tiles(X, tile2)) and (tile2 in connected_tiles(X, tile1)):
        result = True
    return result


def tiles_next(X, tile):
    tiles = [tile_ for tile_ in connected_tiles(X, tile) if is_connected(tile, tile_)]
    return tiles


def go_next(X, tile, tile_previous):
    tiles = tiles_next(X, tile)
    tiles.remove(tile_previous)
    if len(tiles) == 1:
        tile_next = tiles[0]
    else:
        print(f"next tile can be one of {tiles}.")
        return -1
    return tile_next, tile


def split_path(paths):
    n_steps = paths[-1][0]
    for path in paths:
        path[0] = np.min([path[0], n_steps - path[0]])
    return paths
    

def disp_path(X, paths):
    X_path = np.copy(X)
    for path in paths:
        step, tile = path
        X_path[tile[0]][tile[1]] = step
    return X_path

In [116]:
## load data
X = load_data(input_id='')

## find starting position.
paths = []
n_step  = 0
t_start = search_start(X)
print(f"start from {t_start}")
paths.append([n_step, t_start])

## forward one step.
tiles = tiles_next(X, t_start)
#print(f"{t_start} can go to: {tiles}")
tile_previous = t_start
tile = tiles[1]
n_step += 1
paths.append([n_step, tile])
#print(f"{t_start}-->{tile}")

## loop until coming back to the starting point.
while not tile==t_start:
    tile, tile_previous = go_next(X, tile, tile_previous)
    n_step += 1
    paths.append([n_step, tile])
    #print(f"{tile_previous}-->{tile}")

start from [111, 14]


In [117]:
#disp_path(X, paths)

# furthest tile can be reached by 2 ways.
paths = split_path(paths)

# calculate the number of steps to reach the furthest file.
x = [path[0] for path in paths]
print(f"to get from the starting position to the point farthest from the starting position: {np.max(x)}")

6812

6812